In [46]:
import re
from typing import List, Dict
from dataclasses import dataclass

In [47]:
@dataclass
class ConversationMessage:
    speaker: str
    message: str

@dataclass
class AnalysisReport:
    summary: str
    key_actions: List[str]
    sentiment_analysis: Dict[str, dict]

In [48]:
class HindiConversationAnalyzer:
    def __init__(self):
        self.positive_words = {
            'धन्यवाद', 'अच्छा', 'ठीक', 'स्वागत', 'समझते', 'कोशिश', 'उम्मीद', 
            'सहयोग', 'समाधान', 'मदद', 'प्रयास', 'सकारात्मक', 'प्रोत्साहित', 'मित्रतापूर्ण',
            'संतोषजनक', 'प्रगति', 'समर्थन', 'भरोसा', 'विश्वास', 'सम्मान', 'खुशी', 'राहत', 
            'मुलाकात', 'मिलनसार', 'सुरक्षित', 'संतोष', 'सराहना', 'तसल्ली', 'सकारात्मकता'
        }

        self.negative_words = {
            'दिक्कत', 'बुरा', 'नहीं', 'समस्या', 'बकाया', 'नाराज़', 'रोकना', 'तकलीफ़', 
            'हानि', 'क्षति', 'तनाव', 'भ्रम', 'गुस्सा', 'नुकसान', 'चिंता', 'असंतोष', 'निराशा',
            'खतरनाक', 'रुकावट', 'फसाद', 'जटिलता', 'अशांति'
        }

    def parse_conversation(self, text: str) -> List[ConversationMessage]:
        messages = []
        lines = text.strip().split('\n')
        
        for line in lines:
            if line.startswith('Recovery Agent (RA):') or line.startswith('RA:'):
                speaker = 'RA'
                message = line.split(':', 1)[1].strip()
            elif line.startswith('Borrower (B):') or line.startswith('B:'):
                speaker = 'B'
                message = line.split(':', 1)[1].strip()
            else:
                continue
            messages.append(ConversationMessage(speaker=speaker, message=message))
        return messages

    def generate_summary(self, messages: List[ConversationMessage]) -> str:
        has_payment_issue = reason_mentioned = solution_proposed = agreement_reached = False
        summary_points = []
        
        for msg in messages:
            if 'EMI' in msg.message and 'नहीं' in msg.message:
                has_payment_issue = True
            if 'नौकरी' in msg.message and 'चली गई' in msg.message:
                reason_mentioned = True
            if 'आधा भुगतान' in msg.message or 'कुछ भगु तान' in msg.message:
                solution_proposed = True
            if 'ठीक है' in msg.message and msg.speaker == 'B':
                agreement_reached = True
        
        if has_payment_issue:
            summary_points.append("A recovery agent contacted the borrower regarding missed EMI payment.\n")
        if reason_mentioned:
            summary_points.append("The borrower explained job loss as the reason for default.\n")
        if solution_proposed:
            summary_points.append("A partial payment plan was proposed.\n")
        if agreement_reached:
            summary_points.append("Both parties agreed on a revised payment schedule.")
            
        return "".join(summary_points)

    def extract_action_items(self, messages: List[ConversationMessage]) -> List[str]:
        action_items = set()
        
        keyword_actions = {
            ("अगले","हफ्ते", "EMI", "जमा"): "Make partial EMI payment next week",
            ("अगले", "महीने","15", "बाकी" , "भगुतान"): "Clear remaining balance by 15th of  next month",
            ("SMS",): "Check SMS for payment details"
        }
        
        for msg in messages:
            for keywords, action in keyword_actions.items():
                if all(keyword in msg.message for keyword in keywords):
                    action_items.add(action)
        
        return list(action_items)

    def analyze_sentiment(self, messages: List[ConversationMessage]) -> Dict[str, dict]:
        sentiment_scores = {'RA': {'positive': 0, 'negative': 0, 'messages': 0}, 'B': {'positive': 0, 'negative': 0, 'messages': 0}}
        
        for msg in messages:
            sentiment_scores[msg.speaker]['messages'] += 1
            sentiment_scores[msg.speaker]['positive'] += sum(word in msg.message for word in self.positive_words)
            sentiment_scores[msg.speaker]['negative'] += sum(word in msg.message for word in self.negative_words)
        
        return sentiment_scores

    def analyze(self, conversation_text: str) -> AnalysisReport:
        messages = self.parse_conversation(conversation_text)
        summary = self.generate_summary(messages)
        actions = self.extract_action_items(messages)
        sentiment = self.analyze_sentiment(messages)
        
        return AnalysisReport(summary=summary, key_actions=actions, sentiment_analysis=sentiment)

def format_report(report: AnalysisReport) -> str:
    output = ["## Summary", report.summary, ""]
    output.append("## Key Actions")
    output.extend([f"- {action}" for action in report.key_actions])
    output.append("")

    output.append("## Sentiment Analysis")
    for speaker, scores in report.sentiment_analysis.items():
        total_msgs = scores['messages']
        pos, neg = scores['positive'], scores['negative']
        tone = "generally positive" if pos > neg else "generally negative" if neg > pos else "neutral"
        output.extend([
            f"\n{speaker} Analysis:",
            f"- Total messages: {total_msgs}",
            f"- Positive indicators: {pos}",
            f"- Negative indicators: {neg}",
            f"- Overall tone: {tone}"
        ])
    return "\n".join(output)

def analyze_conversation(conversation_text: str) -> str:
    analyzer = HindiConversationAnalyzer()
    report = analyzer.analyze(conversation_text)
    return format_report(report)

In [49]:
conversation_text = """
Recovery Agent (RA): नमस्तेश्री कुमार, मैंएक्स वाई जेड फाइनेंस सेबोल रहा हूं। आपके लोन के बारेमेंबात करनी थी।
Borrower (B): हां, बोलि ए। क्या बात है?
RA: सर, आपका पि छलेमहीनेका EMI अभी तक नहींआया है। क्या कोई समस्या है?
B: हां, थोड़ी दि क्कत है। मेरी नौकरी चली गई हैऔर मैंनया काम ढूंढ रहा हूं।
RA: ओह, यह तो बरुा हुआ। लेकि न सर, आपको समझना होगा कि लोन का भगु तान समय पर करना बहुत जरूरी है।
B: मैंसमझता हूं, लेकि न अभी मेरेपास पसै ेनहीं हैं। क्या कुछ समय मि ल सकता है?
RA: हम समझतेहैंआपकी स्थि ति । क्या आप अगलेहफ्तेतक कुछ भगु तान कर सकतेहैं?
B: मैंकोशि श करूंगा, लेकि न परूा EMI नहींदेपाऊंगा। क्या आधा भगु तान चलेगा?
RA: ठीक है, आधा भगु तान अगलेहफ्तेतक कर दीजि ए। बाकी का क्या प्लान हैआपका?
B: मझु ेउम्मीद हैकि अगलेमहीनेतक मझु ेनया काम मि ल जाएगा। तब मैंबाकी बकाया चकु ा दंगू ा।
RA: ठीक है। तो हम ऐसा करतेहैं- आप अगले हफ्ते तक आधा EMI जमा कर दीजि ए, और अगले महीने के 15 तारीख तक बाकी का भगुतान कर दीजि ए। क्या यह आपको स्वीकार है?
B: हां, यह ठीक रहेगा। मैंइस प्लान का पालन करनेकी परूी कोशि श करूंगा।
RA: बहुत अच्छा। मैंआपको एक SMS भेज रहा हूं जि समेंभगु तान की डि टेल्स होंगी। कृपया इसका पालन करें और समय पर भगु तान करें।
B: ठीक है, धन्यवाद आपके समझनेके लि ए।
RA: आपका स्वागत है। अगर कोई और सवाल हो तो मझु ेबताइएगा। अलवि दा।
B: अलवि दा।
"""


In [50]:

# Run the analysis
result = analyze_conversation(conversation_text)
print(result)

## Summary
A recovery agent contacted the borrower regarding missed EMI payment.
The borrower explained job loss as the reason for default.
A partial payment plan was proposed.
Both parties agreed on a revised payment schedule.

## Key Actions
- Check SMS for payment details
- Make partial EMI payment next week
- Clear remaining balance by 15th of  next month

## Sentiment Analysis

RA Analysis:
- Total messages: 8
- Positive indicators: 5
- Negative indicators: 2
- Overall tone: generally positive

B Analysis:
- Total messages: 8
- Positive indicators: 4
- Negative indicators: 3
- Overall tone: generally positive
